In [1]:
# 디렉토리 내 모든 파일을 리스트로 변환하는 함수 정의

import os

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# 지정된 디렉토리 내 모든 파일명을 리스트로 호출
file_names = list_files('./data')
print(file_names)


['./data/.DS_Store', './data/constitution_of_Korea.pdf']


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_core.prompts import ChatPromptTemplate

# 패키지 관리 (250620)
# conda install conda-forge::langchain-community
# conda install conda-forge::sentence-transformers

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',    # 다국어 모델
    # model_name='jhgan/ko-sroberta-multitask',  # 한국어 모델 - 에러 발생 (250603)
    # model_name = 'BAAI/bge-m3',                # 에러 발생 (250603)
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings_model

/var/folders/kv/jhs4vb392hb9h62z_b83xjwm0000gn/T/ipykernel_85942/3679939528.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [3]:
from langchain.document_loaders import PyMuPDFLoader
# pip install PyMuPDF   # conda 설치 시 에러 발생

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
# conda install conda-forge::langchain-chroma

loader = PyMuPDFLoader(file_names[1])       # 폴더 내 파일 1개만 존재 : 여러 개일 경우, 최초 1개 DB 생성 후, Add 방식으로 진행 (250605)
documents = loader.load()

# 맥의 경우 .ds 파일이 생성되므로 인덱스를 [1]부터 이용한다. (250620)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=16) 
docs = text_splitter.split_documents(documents)

# 임베딩 DB 생성 : 파일로 저장하지 않으므로, 새로 실행할 경우 초기화됨 (250605)
db_constitution = Chroma.from_documents(
    documents=docs, embedding=embeddings_model, collection_name="db_constitution"
)

In [5]:
aa = db_constitution.similarity_search("대통령", k=10)

In [6]:
print(aa[9].page_content, end = "")  # 개행 문자 없이 출력하기 (250620)

제83조 대통령은 국무총리ㆍ국무위원ㆍ행정각부의 장 기타 법률이 정하는 공사의 직을 겸할 수
없다.
 
제84조 대통령은 내란 또는 외환의 죄를 범한 경우를 제외하고는 재직중 형사상의 소추를 받지
아니한다.
 
제85조 전직대통령의 신분과 예우에 관하여는 법률로 정한다.
 
                    제2절 행정부
                       제1관 국무총리와 국무위원

#### Local LLM 사용하기 (250620)

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# pip install langchain-openai


llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    # model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    model = "lmstudio-community/gemma-2-2b-it-GGUF",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # 스트림 출력 콜백
)

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "{input} 한국어로 답변해줘."
)

chain = prompt | llm | StrOutputParser()


# response = chain.invoke("안녕!")
response = chain.invoke({'input' : "안녕!"})

안녕하세요! 😊 무엇을 도와드릴까요? 😄  한국어로 편하게 말씀해주세요. 👍 


In [24]:
# 특정 내용을 검색하고 그 결과를 참고하는 경우,

# 검색 쿼리
query = '탄핵'    # 키워드에 대한 내용을 먼저 추출

# 가장 유사도가 높은 문장 추출
retriever = db_constitution.as_retriever(search_kwargs={'k': 20})
docs = retriever.get_relevant_documents(query)

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = '''Answer the question based only on the following context:
{context} Please answer all the answers in Korean.:

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = prompt | llm | StrOutputParser()

# Chain 실행
query = "대통령은 탄핵될 수 있나요?"
answer = rag_chain.invoke({'context': (format_docs(docs)), 'question': query}) 
# 기 출출된 20개의 context 범위 내에서 question에 대한 응답을 찾을 것

대한민국헌법 제73조에 따르면, 조약을 체결하거나 국제조직에 관한 조약을 체결하는 등의 행위를 통해 대통령이 탄핵될 수 있습니다.  

**더 자세히 설명해 드리겠습니다.**

* **탄핵소추 의결:**  대통령은 탄핵 소추 의결을 받았다면, 그 권한행사가 정지됩니다.
* **탄핵심판:** 탄핵심판이 있을 때까지 대통령의 권한 행사는 정지된다는 것을 의미합니다.
* **탄핵 결정:**  탄핵결정은 공직으로부터 파면되지만, 민사상이나 형사상의 책임이 면제되지 않습니다.


